In [2]:
from pathlib import Path
from sklearn import metrics
import pandas as pd

results = pd.read_csv(Path("./results/results.csv"))

In [68]:
def metrics_to_df(dataset: pd.DataFrame, classifier_name: str, y_pred_label: str):
    y_true = dataset["is_gpt"]
    y_predicted = dataset[y_pred_label]

    df = pd.DataFrame(
        columns=[
            "tn",
            "fp",
            "fn",
            "tp",
            "precision",
            "recall",
            "f1-score",
            "accuracy",
        ],
    )

    confusion_matrix = list(
        metrics.confusion_matrix(y_true=y_true, y_pred=y_predicted).ravel()
    )  # ["tn", "fp", "fn", "tp"]

    precision_recall_fscore_support = list(
        metrics.precision_recall_fscore_support(
            y_true=y_true, y_pred=y_predicted, average="binary"
        )[:-1]
    ) # precision, recall, f1-score
    
    accuracy = metrics.accuracy_score(y_true=y_true, y_pred=y_predicted)

    row = confusion_matrix + precision_recall_fscore_support + [accuracy]

    df.loc[classifier_name] = row

    return df


def metrics_to_csv(dataset: pd.DataFrame):
    # create metrics dataframe

    gptzero_metrics = metrics_to_df(dataset, "gptzero", "gptzero_score_overall")
    orginalityai_metrics = metrics_to_df(dataset, "originalityai", "originality_score_overall")

    csv = pd.concat([gptzero_metrics, orginalityai_metrics])
    csv.to_csv('./results/metrics.csv')

    return csv

classifer_metrics = metrics_to_csv(results)